# Numpy and Astropy FITS

## numpy arrays

Numerical Python (numpy) brings more advanced mathematical and data analysis tools to python.

A core concept in numpy is the numpy array.  You can think of this as similar to a list, but much more advanced.

In [ ]:
import numpy as np

data = np.array([[ 1., 0., 0.],
                 [ 0., 1., 2.]])

In [ ]:
data

In [ ]:
type(data)

In [ ]:
# How many axes does this array have? (i.e. what is the dimension?)
data.ndim

In [ ]:
# What is the "shape" of this array?
data.shape

In [ ]:
# How mnay elements does this array have?
data.size

In [ ]:
# What type of data is in this array?
data.dtype

### Array math

In [ ]:
# Arithmetic is done elementwise
a = np.array([1,2,3,4,5])
b = np.array([11,12,13,14,15])
a+b

In [ ]:
c = [1,2,3,4,5]
d = [11,12,13,14,15]
c + d

In [ ]:
a**2

In [ ]:
np.sin(a) # a is in radians of course

In [ ]:
a<3

In [ ]:
np.where(a<3)

In [ ]:
np.exp(a)

In [ ]:
np.sqrt(a)

### Matrix operations

In [ ]:
A = np.array( [[1,1],
              [0,1]] )
B = np.array( [[2,0],
              [3,4]] )
A*B # elementwise multiplication

In [ ]:
A.dot(B) # dot product

In [ ]:
# alternatively
np.dot(A,B)

### Mixing dtypes

Be careful when mixing dtypes during mathematical operations.  The results follow a complex set of rules for casting one type in to another.

In [ ]:
B = np.array( [[2,0],
              [3,4]], dtype=bool )

In [ ]:
A*B

### Other numpy tools

Numpy has several useful tools for generating arrays.

In [ ]:
# arange is similar to range, but generates a numpy array
np.arange(5)

In [ ]:
np.arange(10,50,5)

In [ ]:
# linspace gives you an array from start to stop, divided in to a number of equal elements
np.linspace(0,np.pi,3)

In [ ]:
# ones and zeros
np.ones((3,5))

In [ ]:
np.zeros((5,3))

In [ ]:
np.zeros((5,3), dtype=bool)

In [ ]:
np.ones((5,3), dtype=float)

# Astropy

"The Astropy Project is a community effort to develop a single core package for Astronomy"

Astropy includes many different tools which may be of use to you including (but not limited to):
* units and constants
* time
* tables
* coordinates
* FITS files and WCS
* models and fitting

In addition Astropy includes the concept of affiliated packages:  "An affiliated package is an astronomy-related Python package that is not part of the astropy core package, but has requested to be included as part of the Astropy project’s community."  Some highlights include:

* ginga FITS viewer
* montage image mosaicing engine
* APLpy (Astronomical Plotting Library in Python)

# astropy.io.fits

The `astropy.io.fits` package evolved from the `pyfits` package which you may have encountered in the past.  `pyfits` has been deprected, though it is being maintained so that it keeps up with `astropy.io.fits` for now.

In [ ]:
from astropy.io import fits

Because `pyfits` was the old standard it is still used in some code.  The `astropy.io.fits` standard evolved from `pyfits` and they were kepts up to date with one another for a while.  To replace `pyfits` you will sometimes see an import statement similar to:

In [ ]:
from astropy.io import fits as pyfits

Astropy provides access to FITS files which requires you to understand a bit more about the FITS file structure than you may have if you've only used FITS viewers in the past.

FITS files are composed of one or more "Header Data Units" (HDUs), so when you open a fits file you will get a list of HDUs.

Each HDU contains a header and data.

In [ ]:
hdul = fits.open('km170918_003.fits')
# hdul = fits.open('kb170805_00050_icuber.fits')
print(type(hdul))

In [ ]:
# This HDU List has only one HDU
hdul

In [ ]:
hdul.info()

In [ ]:
hdu = hdul[0]
hdr = hdul[0].header
hdr

### Headers

In [ ]:
hdr.get('AZ')

In [ ]:
hdr['AZ']

In [ ]:
hdr['FUBAR']

In [ ]:
result = hdr.get('FUBAR')
print(result)

In [ ]:
hdr.comments['AZ']

In [ ]:
for key in hdr.keys():
    print(key)

In [ ]:
hdr.set('AZ', 0.0)

In [ ]:
hdr.get('AZ')

In [ ]:
if not hdr.get('FUBAR'):
    hdr.set('FUBAR', True, 'Situation Normal')

In [ ]:
hdr['FUBAR']

In [ ]:
hdr.comments['FUBAR']

### Data

The data in a FITS image file is read out and stored in a numpy array.

In [ ]:
im = hdul[0].data
print(type(im))

In [ ]:
im.shape

In [ ]:
im

### Conventions Confusion

The numpy arrays which hold the image data are indexable.  Unfortunately, __python and FITS have differing conventions on where the origin is__.

The convention in astronomy (FITS, IRAF, ds9, etc.) is for the origin to be pixel (1,1) and to be in the lower left corner.

In python the origin is (0, 0) because it is zero-based and that is in the upper left corner.

Also, python ordering is (row, column) while many astronomical tools order it (x, y) or (column, row).

This can be __very confusing__. Expecially when taking pixel coordinates from FITS headers. :(

In [ ]:
# Here are the values of the upper right corner of the above array
# (which would be displayed in the lower left if you were using IRAF or ds9)
im[0:2,0:3]
# Note that I've printed out rows 0:2 and columns 0:3.

### FITS Tables

Astropy also supports FITS tables, but we will leave those for you to examine on your own.  As always, information is on the astropy documentation page:
http://docs.astropy.org

### Manipulating FITS Files

In [ ]:
hdul = fits.open('km170918_003.fits')
hdul[0].header.get('FUBAR')

In [ ]:
if not hdul[0].header.get('FUBAR'):
    hdul[0].header.set('FUBAR', True, 'Situation Normal')

import os
if os.path.exists('newfits.fits'):
    os.remove('newfits.fits')
hdul.writeto('newfits.fits', output_verify='ignore')

In [ ]:
hdul2 = fits.open('newfits.fits')
hdul2[0].header.get('FUBAR')

### Closing FITS Files

FITS files can take up lots of memory if they contain a lot of pixels.  If you're trying to open a single large file, there is an option (`mmap`) which will tell astropy not to read the entire file in to memory.

A more common situation is when you have many files you are opening.  Once you use a file, you may not need it's data after a certain point, so there is no reason for it to remain in memory.  Normally, I would just rely on python's built in memory cleanup to deal with this, but looping through numerous FITS files can overfill your memory.  

The solution is to use the `.close()` method on an `HDUList` when you are done with it, but it is ught to have many `hdul.close()` scattered throughout your code, a cleaner, easier to read solution is to use python's `with`:

In [ ]:
# hdul3 = fits.open('newfits.fits', mode='update')
# hdul3[0].header.set('FUBAR', False)
# hdul3.flush(output_verify='ignore')
# hdul3.close()

with fits.open('newfits.fits', mode='update') as hdul3:
    hdul3[0].header.set('FUBAR', False)
    hdul3.flush(output_verify='ignore')

In [ ]:
# hdul4 = fits.open('newfits.fits', mode='readonly')
# fubar = hdul4[0].header.get('FUBAR')
# print(fubar)
# hdul4.close()

with fits.open('newfits.fits', mode='readonly') as hdul4:
    fubar = hdul4[0].header.get('FUBAR')
print(fubar)

# Astropy Command Line Utilities

For convenience, several of Astropy’s subpackages install utility programs on your system which allow common tasks to be performed without having to open a Python interpreter. These utilities include:

* fitsheader: prints the headers of a FITS file.
* fitscheck: verifies and optionally re-writes the CHECKSUM and DATASUM keywords of a FITS file.
* fitsdiff: compares two FITS files and reports the differences.
* Scripts: converts FITS images to bitmaps, including scaling and stretching.
* wcslint: checks the WCS keywords in a FITS file for compliance against the standards.

# World Coordinate Systems

Astropy includes support for world coordinate system (WCS) representations.  The WCS is a very powerful, generic system for describing the transformation between pixel coordinates and some other system (e.g. RA, Dec).  Because it is so powerful and generic, it is also complicated.

In [ ]:
from astropy.wcs import WCS
import numpy as np

In [ ]:
w = WCS(hdul[0].header)

In [ ]:
w

In [ ]:
# Three pixel coordinates
pix = np.array([[0, 0],
                [24, 38],
                [45, 98]])
# Convert to world coordinates
world = w.wcs_pix2world(pix, 1)
world

In [ ]:
# Convert back to pixels
pix2 = w.wcs_world2pix(world, 1)
pix2

In [ ]:
pix3 = np.array(np.round(pix2), dtype=int)
pix3

The `,1` in the conversions above is to specify the origin.  Here, origin is the coordinate in the upper left corner of the image. In FITS and Fortran standards, this is 1. In Numpy and C standards this is 0.

Astropy WCS supports SIP distortions and many projections.  See the astropy documentation for more info on the details.

# Exercises

### Exercise 1) Verify that your file as a WCS

Open the file `km170920_002.fits`.

First, verify that the file has a WCS and print a message if it has one.

In [ ]:
# Your code here

### Exercise 2) RA, Dec to Pixel Coordinates

There is a star of interest at coordinates `(RA, Dec) = (271.82875, 66.47797)`.  This should be in your field of view.  Using the WCS in the image, determine which pixel this coordinate corresponds to.

Using formatted printing, print the result in the form:
```
xpixel = 1023.45
ypixel =   67.89
```

In [ ]:
# Your code here

### Exercise 3) Grab subsection of the Image

Get a numpy array that covers a 8x8 pixel region centered on the pixel position of the star.  Print it to the screen and visually confirm that the numbers near the center have large values (>10,000 ADU).

Hint: you will be subscripting a numpy array which requires that the subscripts be integer values, not float.

In [ ]:
# Your code here